# miniedit distance and overlap analysis

In [73]:
def minDistance(word1: str, word2: str) -> int:
    n = len(word1)
    m = len(word2)
    
    # 有一个字符串为空串
    if n * m == 0:
        return n + m
    
    # DP 数组
    D = [ [0] * (m + 1) for _ in range(n + 1)]
    
    # 边界状态初始化
    for i in range(n + 1):
        D[i][0] = i
    for j in range(m + 1):
        D[0][j] = j
    
    # 计算所有 DP 值
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            left = D[i - 1][j] + 1
            down = D[i][j - 1] + 1
            left_down = D[i - 1][j - 1] 
            if word1[i - 1] != word2[j - 1]:
                left_down += 1
            D[i][j] = min(left, down, left_down)
    
    return D[n][m]

In [74]:
string1="猜猜我是谁"
string2="我是z谁c"
'''
operation:
插入一个字符；
删除一个字符；
替换一个字符。
'''

print(minDistance(string1,string2))

4


In [137]:
import json,os
import numpy as np
import jieba
import re

In [76]:
query_p="./query.json"
querydic={}
with open(query_p,"r") as fr:
    lines=fr.readlines()
    for line in lines:
        entry=json.loads(line)
        rid=entry["ridx"]
        query=entry["q"]
        querydic[rid]=query

# 1.Word extraction
本节查看chatgpt做词语抽取，有多少内容是chatgpt补充的，或者与原文的词语不匹配的。

In [239]:
wordls=["./reduced query/chat_word2.json"]
wordqdict={}
for word_p in wordls:
    with open(word_p,"r") as fr:
        lines=fr.readlines()
        for line in lines:
            entry=json.loads(line)
            rid=entry["ridx"]
            query=entry["q"][4:]
            wordqdict[int(rid)]=query

In [240]:
wordkeys=list(wordqdict.keys())
idx=wordkeys[0]
print(wordqdict[idx])

酒后驾车、乙醇含量、抽取血样、刑事责任能力、自首。


In [241]:
annot_root="./annot"
annotdic={}
files=os.listdir(annot_root)
for file in files:
    path=os.path.join(annot_root,file)
    with open(path,"r") as fr:
        entry=json.load(fr)
        qid=entry["task_id"]
        q=entry["html"]
        annotdic[qid]=q

In [242]:
print(annotdic)

{-1071: '一、2005年，被告人董国华利用其担任北京市文化设施建设管理中心（以下简称设施中心）主任的职务便利，以支付“宿舍楼维修费”的名义<span>骗取单位公款人民币84万余元</span>（以下币种均为人民币），用于个人购买北京市西城区房屋，后使用虚假开具的发票用于单位平账。案发后，该房产已被查封。二、2006年至2007年间，被告人董国华伙同被告人高岭利、焦守江，利用董国华担任设施中心及北京市文化局机关事务管理服务中心（以下简称服务中心）主任的职务便利，<span>签订虚假协议，以支付物业费、供暖费、工地接管费等名义分四次套取单位公款共计224万余元</span>，并将上述钱款转入三名被告人控制的北京文华华泰文化发展有限公司（以下简称文华华泰公司）账户，后将部分款项用于支付供暖费、物业费和保安费，剩余159万余元被三名被告人非法占有。三、2007年至2012年间，被告人董国华伙同王某1（已判刑），利用董国华担任设施中心主任的职务便利，虚构设施中心与北京环新健海商贸有限责任公司（以下简称环新健海公司）存在合作投资的事实，<span>伪造“投资协议书”和“投资款收据”等材料，</span>后董国华以<span>归还上述投资款的名义，骗取单位公款283万余元</span>，转入王某1控制的环新健海公司账户并非法占有，其中部分赃款用于董国华购买北京市海淀区房产。案发后，该套房产已被查封，王某1向司法机关退缴赃款36万元。四、2001年至2006年间，被告人董国华利用其担任设施中心主任的职务便利，为被告人焦守江承揽文化局及相关单位的工程项目提供帮助，董国华于2003年向焦守江索要现金30万元，用于其子购买北京市西城区房屋。案发后，该套房产已被查封。五、2008年至2009年间，被告人董国华利用其担任服务中心主任的职务便利，为北京东方景丽酒店物业管理有限公司（以下简称东方景丽公司）承包北京市文化局下属单位物业管理项目提供帮助，并以代为销售外国化妆品的名义，向该公司负责人王某1<span>多次索要现金共计12万元。</span>综上，被告人董国华单独或伙同被告人高岭利、焦守江<span>贪污数额共计527万余元、受贿数额共计42万元</span>，被告人高岭利、焦守江参与贪污159万余元，焦守江向董国华行贿30万元。2016年6月20日，被告人董国华被抓获归案。同年

In [243]:
for qid in wordqdict:
    query=wordqdict[qid]
    originalq=querydic[qid]
    items=query.split("、")
    count=0
    notappear=[]
    for item in items:
        if item not in originalq:
            count+=1
            notappear.append(item)
    print("qid:{},not appear:{},total:{}".format(qid,count,len(items)))
    print(notappear)

qid:5156,not appear:2,total:5
['酒后驾车', '自首。']
qid:4891,not appear:0,total:13
[]
qid:5187,not appear:3,total:7
['酒后驾车', '拒捕', '乙醇含量。']
qid:330,not appear:1,total:19
['修理费。']
qid:706,not appear:2,total:13
['卖出', '被盗物品的价值。']
qid:259,not appear:1,total:6
['罚金。']
qid:221,not appear:1,total:16
['18K彩金耳钉。']
qid:2132,not appear:1,total:12
['刑事责任。']
qid:2143,not appear:2,total:15
['赔偿款项', '营养等费用。']
qid:1972,not appear:1,total:8
['约架']
qid:1978,not appear:1,total:6
['人民币23000元。']
qid:2361,not appear:1,total:11
['判决。']
qid:2373,not appear:4,total:11
['致死', '酒驾', '无证驾驶', '不戴头盔。']
qid:2331,not appear:3,total:7
['醉酒驾驶', '主次责任', '判决参考价值。']
qid:3228,not appear:2,total:12
['售卖', '行政拘留。']
qid:3746,not appear:2,total:10
['公安机关抓捕', '当场抓获。']
qid:3765,not appear:5,total:9
['贩卖毒品', '购买毒品', '容留他人吸食毒品', '查获毒品及制毒工具', '枪支及子弹。']
qid:3342,not appear:1,total:10
['扣押。']
qid:1405,not appear:2,total:19
['虚构信息', '骗取钱财']
qid:1430,not appear:2,total:9
['公章伪造', '犯罪事实交代']
qid:1325,not appear:3,total:12
['法律案件', '判决', '重要参考

In [244]:
def labelgen(annotation):
    '''
        inputs:
            annotation:标注的原文内容，即"<html></html>标签内的文本"
        func:
            对标注
        outputs:
            对原始query的标注序列,长度与query一致;0为未标注的字符,1为标注的字符;
    '''
    # 0为在span外,1为在span内;注意我们包括标点;
    result=annotation.split("<span>")
    labels=[]
    tmp=[0 for _ in range(len(result[0]))]
    labels.extend(tmp)
    for item in result[1:]:
        itemls=item.split("</span>")
        tmp=[1 for _ in range(len(itemls[0]))] # 第一个是在<span> </span>中的
        labels.extend(tmp)
        for item in itemls[1:]:
            tmp=[0 for _ in range(len(item))] # 其它不是;
            labels.extend(tmp)
    return labels

In [245]:
def annot_ls(annotation):
    '''
        funcs:
            生成标注的内容列表;
    '''
    # 0为在span外,1为在span内
    result=annotation.split("<span>")
    labels=[]
    for item in result[1:]:
        itemls=item.split("</span>")
        labels.append(itemls[0])
    return labels

In [248]:
# labellsdic={}
recalldic={} # 计算overlap
com_Rdic={} # 计算“抽取”压缩率

for qid in annotdic:
    annot=annotdic[qid]
    annotls=annot_ls(annot) # 获取
    # print(res)
    query=wordqdict[qid] # 词语抽取后的query;
    originalq=querydic[qid]
    qwordset=set(jieba.cut(originalq, cut_all=False)) # 原始query的不重合词语数

    items=query.split("、") #chatgpt抽取后的词语用、分割
    itemset=set()
    for i in items:
        iwords=jieba.cut(i, cut_all=False)
        itemset=itemset.union(set(iwords))

    # 由于生成模型做抽取任务会不准确，但是生成模型本身比较强
    # 因此用编辑距离匹配生成模型的抽取与标注内容
    # 考虑召回(overlap)：
    # 对每一个标注的词语，寻找抽取内容的最小编辑距离内容，作为召回匹配,观察overlap
    overlapcount=0
    totalcount=0
    for ant in annotls:
        antwordls=jieba.cut(ant, cut_all=False)
        for antword in antwordls:
            minindex=-1
            minloss=100000
            for idx,item in enumerate(items):
                tmp=minDistance(item,antword)
                if tmp < minloss:
                    minindex=idx
                    minloss=tmp
            matchedword=items[minindex]
            totalcount+=len(matchedword)
            # 字符级别召回,因为生成的词语基本流畅;
            itemchr=list(item)
            antchar=list(ant)
            for atchar in antchar:
                if atchar in itemchr:
                    overlapcount+=1
    recalldic[qid]=overlapcount/totalcount
    itemlen=0
    for i in itemset:
        itemlen+=len(i)
    qwordlen=0
    for qw in qwordset:
        qwordlen+=len(qw)
    com_Rdic[qid]=(overlapcount/totalcount)/(itemlen/qwordlen)

# entries=np.array(list(recalldic.values()))
# print(np.average(entries)) # 但是annotation的粒度一般比较大;

entries=np.array(list(recalldic.values()))
print("ComRecall Ave:{}".format(np.average(entries)))

entries=np.array(list(com_Rdic.values()))
print("infoR Ave:{}".format(np.average(entries))) # infoR,笔误;
# word1:0.3446,66.81,1.419
# word2:0.3224,38.79,2.037

ComRecall Ave:0.3224546859510307
ComRecall Ave:2.0371598873167818


# 2.sentence extraction analysis

In [138]:
def split_sentences(line):
    '''
        依据大标点处理文本，分割得到句子;
    '''
    line_split = re.split(r'[。！；？，]',line.strip())
    line_split = [line.strip() for line in line_split if line.strip() not in ['。','！','？','；','，'] and len(line.strip())>1]
    return line_split


In [150]:
def span_supple(annotsentls):
    '''
        由于句子用大标点分割后，其中一些span是跨句子的，因此需要进行补全;
    '''
    newls=[]
    missflag=False # 表示有一个左标签没有闭合，那么其中的item都是标注;
    for item in annotsentls:
        bfirst=item.find('<span>')
        rfirst=item.find('</span>') # 左标签第一次出现，右标签第一次出现位置;
        if bfirst ==-1 and rfirst==-1 and not missflag:
            newls.append(item)
            continue
        if bfirst ==-1 and rfirst==-1 and missflag:
            newls.append('<span>'+item+'</span>')
            continue
        # 有一个标签出现过;
        if bfirst ==-1:
            # 只有右标签，找不到左标签,最左边补上一个<span>;
            newls.append('<span>'+item)
            missflag=False
            continue
        elif rfirst==-1:
            # 只有左标签，找不到右标签，最右边补上一个标签'</span>';
            newls.append(item+'</span>')
            missflag=True
            continue
        
        blast=item.rfind('<span>')
        rlast=item.rfind('</span>')      # 左标签最后一次，右标签最后一次出现的位置;

        if blast > rlast:
            # 左标签没有闭合;
            newls.append(item+'</span>')
            missflag=True
            continue
        newls.append(item)
    if len(newls) !=len(annotsentls):
        print("error")
    return newls

In [167]:
def span_ex(e):
    '''
        依据补全后的切分的句子,获取标签
    '''
    labels=[]
    bindex=e.find('<span>')
    if bindex ==-1:
        # 没有左span
        eindex=e.find('</span>')
        if eindex==-1:
        # 没有右span,则没有span
            return labels
        # 有右span
        result=e.split("</span>")
        labels.append(result[0])
        return labels
    # 有左span
    result=e.split("<span>")
    if len(result)==1:
        # <span>在开头
        for item in result[0]:
            itemls=item.split("</span>")
            labels.append(itemls[0])
    else:
        # <span>在中间
        for item in result[1:]:
            # 每一个<span>后面,都有0/1个</span>
            itemls=item.split("</span>")
            labels.append(itemls[0])
    return labels
        

In [159]:
# 测试
for k,v in annotdic.items():
    tmp=split_sentences(v)
    result=span_supple(tmp)
    # print(k)
    # print(result)
    labels=span_ex(result)
    # print(labels)

In [ ]:
sentls=["./reduced query/chat_sent1.json"]
sentqdict={}
for sent_p in sentls:
    with open(sent_p,"r") as fr:
        lines=fr.readlines()
        for line in lines:
            entry=json.loads(line)
            rid=entry["ridx"]
            query=entry["q"]
            sentqdict[int(rid)]=query

In [139]:
print(sentqdict)

{5156: '2018年1月15日14时10分许，被告人莫新国酒后驾驶湘A×××××号小型轿车沿长沙市天心区伊莱克斯大道由南往北行驶至水电八局基地路段时被在该处执勤的长沙市公安局交通警察支队民警检查，经现场酒精吹气检测，测试结果显示其血液中乙醇含量为195毫克／100毫升。\n被告人莫新国被交警带至湖南省融城医院抽取血样，并将血样送至长沙市公安局物证鉴定所检验，经检验，其血液中乙醇含量为201.1毫克／100毫升。\n经湖南省芙蓉司法鉴定中心鉴定，被告人莫新国作案时处于普通醉酒状态，实施危害行为时有完全刑事责任能力。\n2018年1月30日，被告人莫新国主动到公安机关投案，其归案后如实供述了自己的罪行。', 4891: '孙小寨酒后驾驶机动车辆，属于醉酒状态。\n孙小寨阻碍和辱骂民警，属于妨碍公务罪。\n孙小寨持刀威胁和辱骂他人，属于寻衅滋事罪。\n孙小寨投案并如实交代犯罪事实。', 5187: '被告人王X酒后无证驾驶云GS0708号二轮摩托车行至弥勒市朋普镇情缘网吧路段时。\n公安民警阮X示意其停车接受检查，被告人王X没有停车接受检查。\n被告人王X非但不配合公安民警检查，还对公安民警进行辱骂。\n被告人王X1、李X见状后便上前和被告人王X一起辱骂公安民警，并用拳脚对公安民警阮X、田X、陈X等人进行殴打。\n经云通司法鉴定中心鉴定，田X、阮X、陈X此次外伤损伤程度为轻微伤。\n被告人王X血液中检出乙醇成分，其乙醇含量为106mg/100ml血。', 330: '被告人杨海、陈洪凯盗窃行车记录仪\n被抓获在摩托车内查获所盗行车记录仪\n被盗行车记录仪已追回发还失主\n被告人陈洪凯盗窃时驾驶的摩托车被查扣\n被告人陈洪凯家属赔偿被害人张某1车窗修理费', 706: '被告人张某某、王某某、雷某某、周某某、龙某某、李某某、牛某某分别结伙驾车携带钢筋钳、砍刀、消防枪等工具，多次窜至汤阴县、安阳县、内黄县、安阳市文峰区、河北省临漳县、河北省魏县、河北省磁县、河北省武安市等地盗窃通信电缆、电瓶、电动车等物品。\n张某某参与盗窃十三次，盗窃物品价值243293元；王某某、雷某某均参与盗窃十二次，盗窃物品价值241853元；周某某参与盗窃八次，盗窃物品价值142516元；龙某某参与盗窃五次，盗窃物品价值79153元；李某某参与盗窃三次，盗窃物品价值66182元；牛某某参与盗窃

## 2.1 原始query按照句号切分
标注内容直接按照原始标注切分

In [250]:
# labellsdic={}
sent_Rdic={} # Recall
com_Rdic={} # InfoR
rqleng=0
for qid in annotdic:
    annot=annotdic[qid]
    annotls=annot_ls(annot) # seperate annotation;
    # print(res)
    query=sentqdict[qid]
    originalq=querydic[qid]
    olditems=query.split("。")
    # olditems=split_sentences(query)
    items=[]
    for item in olditems:
        if item=="":
            continue
        tmpitem=item.strip("\\n")
        items.append(tmpitem)
        rqleng+=len(tmpitem)

    overlapcount=0
    totalcount=0
    for ant in annotls:
        minindex=-1
        minloss=100000
        for idx,item in enumerate(items):
            tmp=minDistance(item,ant)
            if tmp < minloss:
                minindex=idx
                minloss=tmp
        matchedword=items[minindex]
        totalcount+=len(ant)
        itemchr=list(matchedword)
        antchar=list(ant)
        for atchar in antchar:
            if atchar in itemchr:
                overlapcount+=1
    if totalcount==0:
        print(totalcount)
    sent_Rdic[qid]=overlapcount/totalcount
    com_Rdic[qid]=(overlapcount/totalcount)/ (len(query)/len(originalq))

entries=np.array(list(sent_Rdic.values()))
print("ComRecall Ave:{}".format(np.average(entries)))
print("Recall length:{}".format(rqleng/len(annotdic)))

entries=np.array(list(com_Rdic.values()))
print("ComRecall Ave:{}".format(np.average(entries)))

ComRecall Ave:0.5226050766271143
Recall length:219.44859813084113
ComRecall Ave:1.072237034196783


### 标注按全标点切分

In [ ]:
# labellsdic={}
sent_Rdic={}
rqleng=0
for qid in annotdic:
    annot=annotdic[qid]
    tmp=split_sentences(annot)
    annotdivide=span_supple(tmp)
    # print(res)
    query=sentqdict[qid]
    olditems=query.split("。")
    # olditems=split_sentences(query)
    items=[]
    for item in olditems:
        if item=="":
            continue
        tmpitem=item.strip("\\n")
        items.append(tmpitem)
        rqleng+=len(tmpitem)

    overlapcount=0 # 一个query的overlap
    totalcount=0 # 一个query的标注的totalcount
    for antsent in annotdivide:
        # 对每个原始句子匹配
        if "<span>" not in antsent and "</span>" not in antsent:
            continue
        # 注意匹配时去除标记;
        msg = antsent.replace('<span>', '')
        msg2 = msg.replace('</span>', '')

        minindex=-1
        minloss=100000
        for idx,item in enumerate(items):
            tmp=minDistance(item,msg2)
            if tmp < minloss:
                minindex=idx
                minloss=tmp
        matchedword=items[minindex]
        # 再把annotated sentence中的标注取出来；
        antls=span_ex(antsent)
        for antword in antls:
            totalcount+=len(antword)
            itemchr=list(matchedword)
            antchar=list(antword)
            for atchar in antchar:
                if atchar in itemchr:
                    overlapcount+=1
        if totalcount==0:
            print(totalcount)
            print(annotdivide)
    sent_Rdic[qid]=overlapcount/totalcount

entries=np.array(list(sent_Rdic.values()))
print(np.average(entries)) 
print("Ave:{}".format(rqleng/len(annotdic)))


0.5452763924749808
Ave:219.44859813084113


# 3.Summary analysis

In [220]:
sumls=["./reduced query/chat_sum2.json"]
sumqdict={}
rqleng=0
for sum_p in sumls:
    with open(sum_p,"r") as fr:
        lines=fr.readlines()
        for line in lines:
            entry=json.loads(line)
            rid=entry["ridx"]
            query=entry["q"]
            sumqdict[int(rid)]=query

In [221]:
print(sumqdict)

{5156: '被告人莫新国醉酒驾车被民警查获，血液中乙醇含量高于饮酒后行车规定。被告人莫新国被鉴定为精神残疾人，但在作案时具有完全刑事责任能力。被告人莫新国投案自首，并如实供述罪行。', 4891: '被告人孙小寨于2018年8月24日酒后驾驶车辆，涡阳县公安局民警接警后到达现场，孙小寨被检验出血液酒精含量151.2503mg／100ml。随后，孙小寨在涡阳县陈大镇姜洼行政村孙黑楼自然村阻碍民警拍照并进行威胁，并在涡阳县医院急救中心对公安局工作人员进行辱骂。案发后，孙小寨主动到公安机关投案，并如实交代了其妨碍公务的犯罪事实。', 5187: '被告人王X无证驾驶摩托车，并拒绝接受检查。公安民警阮X、田X追上并控制住王X时，王X及其同伴李X对民警进行辱骂和殴打。结果，田X、阮X和陈X受到了轻微伤。被告人王X血液中检出乙醇成分。', 330: '被告人杨海、陈洪凯驾驶摩托车盗窃丰田轿车内的捷渡行车记录仪。被抓获时在摩托车内查获所盗物品，被盗车内损坏已赔偿，陈洪凯盗窃时驾驶的摩托车被查扣。', 706: '本案涉及多名被告人结伙盗窃通信电缆、电瓶、电动车等物品，并将部分物品出售给他人获取非法利益。其中被告人张某某、王某某、雷某某、周某某、龙某某、李某某、牛某某分别参与盗窃多次，盗窃物品总价值超过60万元；被告人朱某某、赵某某、李某甲明知所购买的通信电缆是盗窃所得，但仍进行六次收购，总价值为12.1万元。此案可供类案检索参考。', 259: '本案为非法占有纯碱案，被告人刘某甲多装载货物，被告人汪某甲、程某、兰某、汪某乙参与协助，案发后被告人刘某甲和刘某乙退还部分非法所得。涉案金额人民币19650元。被告人刘某乙主动缴纳罚金二万元。', 221: '薛亮和陈某预谋盗窃，薛亮独自进入顾某住宅盗取大量贵重物品，包括黄金饰品、电脑、项链等。涉案物品估价高达数万元。', 2132: '被告人陈建军因与他人发生争执，在烧烤店门口使用折叠刀捅刺两名被害人，造成重伤二级。经公安局鉴定，被害人蔡某和王某分别遭到多处创口。被告人陈建军在被抓获时携1把折叠刀。', 2143: '（一）巴中鑫金地实业有限公司通过公开竞拍取得了巴州区回风津桥湖片区国有建设用地使用权，开发建设“英伦庄园”项目。李刚等人为该项目用地范围内的村民，阻挠了工程建设方的施工，导致经济损失124530元。\n\n（二）李刚

## 全标点切分标注、原始句子

In [222]:
# labellsdic={}
sum_Rdic={} # overlap
com_Rdic={} # compression rate
rqleng=0
for qid in annotdic:
    annot=annotdic[qid]
    tmp=split_sentences(annot)
    annotdivide=span_supple(tmp)
    originalq=querydic[qid]
    # print(res)
    query=sumqdict[qid]
    items=split_sentences(query)
    # items=query.split("，")
    for item in items:
        rqleng+=len(item)

    overlapcount=0 # 一个query的overlap
    totalcount=0 # 一个query的标注的totalcount
    for antsent in annotdivide:
        # 对每个原始句子匹配
        if "<span>" not in antsent and "</span>" not in antsent:
            continue
        # 注意匹配时去除标记;
        msg = antsent.replace('<span>', '')
        msg2 = msg.replace('</span>', '')

        minindex=-1
        minloss=100000
        for idx,item in enumerate(items):
            tmp=minDistance(item,msg2)
            if tmp < minloss:
                minindex=idx
                minloss=tmp
        matchedword=items[minindex]
        # 再把annotated sentence中的标注取出来；
        antls=span_ex(antsent)
        for antword in antls:
            totalcount+=len(antword)
            itemchr=list(matchedword)
            antchar=list(antword)
            for atchar in antchar:
                if atchar in itemchr:
                    overlapcount+=1
        if totalcount==0:
            print(totalcount)
            print(annotdivide)
    sum_Rdic[qid]=overlapcount/totalcount
    com_Rdic[qid]=(overlapcount/totalcount)/ (len(query)/len(originalq))

entries=np.array(list(sum_Rdic.values()))
print("ComRecall Ave:{}".format(np.average(entries)))
print("Recall length:{}".format(rqleng/len(annotdic)))

entries=np.array(list(com_Rdic.values()))
print("ComRecall Ave:{}".format(np.average(entries)))

# 全标点切分;选用此

# sum1
# Recall Ave:0.4399085175489412
# Recall length:120.92523364485982
# ComRecall Ave:1.4094868523863449

# sum2
# 0.4624564196603436
# Ave:132.73831775700936
# ComRecall Ave:1.3430513473441834

# 逗号切分：

# sum1
# 0.35855721943482094
# Ave:124.29906542056075

# sum2
# 0.3966534040243279
# Ave:136.3271028037383



ComRecall Ave:0.4624564196603436
Recall length:132.73831775700936
ComRecall Ave:1.3430513473441834


## 用逗号切分
未选用

In [134]:
# labellsdic={}
sum_Rdic={}
rqleng=0
for qid in annotdic:
    annot=annotdic[qid]
    annotls=annot_ls(annot)
    # print(res)
    query=sumqdict[qid]
    originalq=querydic[qid]
    items=query.split("，")
    for item in items:
        rqleng+=len(item)
    overlapcount=0
    totalcount=0
    for ant in annotls:
        minindex=-1
        minloss=100000
        for idx,item in enumerate(items):
            tmp=minDistance(item,ant)
            if tmp < minloss:
                minindex=idx
                minloss=tmp
        matchedword=items[minindex]
        totalcount+=len(ant)
        itemchr=list(matchedword)
        antchar=list(ant)
        for atchar in antchar:
            if atchar in itemchr:
                overlapcount+=1
    sum_Rdic[qid]=overlapcount/totalcount
entries=np.array(list(sum_Rdic.values()))
print(np.average(entries)) #0.19->0.23
print("Ave:{}".format(rqleng/len(annotdic)))

# sum1:
# 0.2998
# 124.29

# sum2:
# 0.3157
# 136.33

0.31579456600015704
Ave:136.3271028037383


In [135]:
# labellsdic={}
sum_Rdic={}
rqleng=0
for qid in annotdic:
    annot=annotdic[qid]
    annotls=annot_ls(annot)
    # print(res)
    query=sumqdict[qid]
    originalq=querydic[qid]
    items=query.split("，")
    for item in items:
        rqleng+=len(item)
    overlapcount=0
    totalcount=0
    for ant in annotls:
        antwordls=jieba.cut(ant, cut_all=False) 
        for antword in antwordls:
            minindex=-1
            minloss=100000
            for idx,item in enumerate(items):
                tmp=minDistance(item,antword)
                if tmp < minloss:
                    minindex=idx
                    minloss=tmp
            matchedword=items[minindex]
            totalcount+=len(antword)
            itemchr=list(matchedword)
            antchar=list(antword)
            for atchar in antchar:
                if atchar in itemchr:
                    overlapcount+=1
    sum_Rdic[qid]=overlapcount/totalcount
entries=np.array(list(sum_Rdic.values()))
print(np.average(entries)) #0.19->0.23
print("Ave:{}".format(rqleng/len(annotdic)))

# sum1:
# 0.1580
# 124.29

# sum2:
# 0.16
# 136.33

0.1621717531245073
Ave:136.3271028037383
